In [ ]:
from IPython.display import display, Javascript, HTML
import ipywidgets as ipw
from src import widgets, utils
import json

In [ ]:
CONFIG_FILENAME = "config/config.json"
CONFIG = utils.read_json(CONFIG_FILENAME)
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis_aiida()

In [ ]:
OPENBIS_COLLECTIONS_PATHS = utils.read_json("metadata/collection_paths.json")

In [ ]:
if OPENBIS_SESSION:
    increase_buttons_size = HTML(data="".join(CONFIG["save_home_buttons_settings"]))
    create_substances_vbox = widgets.CreateSubstanceWidget(OPENBIS_SESSION)

home_button = ipw.Button(
    description="",
    disabled=False,
    button_style="",
    tooltip="Main menu",
    icon="home",
    layout=ipw.Layout(width="100px", height="50px"),
)

save_button = ipw.Button(
    description="",
    disabled=False,
    button_style="",
    tooltip="Save",
    icon="save",
    layout=ipw.Layout(width="100px", height="50px"),
)

save_close_buttons_hbox = ipw.HBox([save_button, home_button])

In [ ]:
def close_notebook(b):
    display(Javascript(data='window.location.replace("home.ipynb")'))


def save_substance(b):
    substance_dict = {
        "name": create_substances_vbox.name_textbox.value,
        "description": create_substances_vbox.description_textbox.value,
        "empa_number": create_substances_vbox.empa_number_textbox.value,
        "batch": create_substances_vbox.batch_textbox.value,
        "vial": create_substances_vbox.vial_textbox.value,
        "purity": create_substances_vbox.purity_textbox.value,
        "substance_type": create_substances_vbox.substance_type_textbox.value,
        "chemist_own_name": create_substances_vbox.chemist_own_name_textbox.value,
        "special_storage_conditions": list(
            create_substances_vbox.storage_conditions_selector.value
        ),
        "object_status": create_substances_vbox.object_status_dropdown.value,
        "synthesised_by": list(create_substances_vbox.synthesised_by_selector.value),
        "supplier_own_name": create_substances_vbox.supplier_own_name_textbox.value,
        "comments": create_substances_vbox.comments_textbox.value,
    }

    amount_value = create_substances_vbox.amount_value_textbox.value
    if amount_value:
        amount_unit = create_substances_vbox.amount_unit_dropdown.value
        amount_quantity_value = {"value": amount_value, "unit": amount_unit}
        substance_dict["amount"] = json.dumps(amount_quantity_value)

    location_value = create_substances_vbox.location_dropdown.value
    if location_value != "-1":
        substance_dict["location"] = location_value

    package_opening_date = create_substances_vbox.package_opening_date.value
    if package_opening_date:
        substance_dict["package_opening_date"] = package_opening_date

    supplier_value = create_substances_vbox.supplier_dropdown.value
    if supplier_value != "-1":
        substance_dict["supplier"] = supplier_value

    receive_date = create_substances_vbox.receive_date.value
    if receive_date:
        substance_dict["receive_date"] = receive_date

    molecules_ids = []
    for molecule_widget in create_substances_vbox.molecules_accordion.children:
        molecule_id = molecule_widget.dropdown_list.value
        if molecule_id != "-1":
            molecules_ids.append(molecule_id)

    substance_dict["molecules"] = molecules_ids

    evaporation_temperatures_table = (
        create_substances_vbox.evaporation_temperatures_table.table_gridbox.children
    )
    num_cells_row = 4
    num_rows = len(evaporation_temperatures_table) // num_cells_row
    ob_spreadsheet = OPENBIS_SESSION.new_spreadsheet()
    ob_spreadsheet_style = {}
    ob_spreadsheet_data = []
    ob_spreadsheet_headers = ["Datetime", "Instrument name", "Temperature"]
    ob_spreadsheet_meta = None
    ob_spreadsheet_width = ["100", "100", "100"]
    ob_spreadsheet_values = []

    for row in range(1, num_rows):
        row_cells = evaporation_temperatures_table[
            num_cells_row * row : num_cells_row * (row + 1)
        ]
        datetime_value = row_cells[0].value
        if datetime_value:
            datetime_value = datetime_value.strftime("%Y/%m/%d")
        else:
            datetime_value = ""
        instrument_name = row_cells[1].value
        temperature_data = {"value": row_cells[2].value, "unit": row_cells[3].value}
        temperature_data = json.dumps(temperature_data)
        row_data = [datetime_value, instrument_name, temperature_data]

        ob_spreadsheet_data.append(row_data)
        ob_spreadsheet_values.append(row_data)

        ob_spreadsheet_style[f"A{row}"] = "text-align: center; overflow: hidden;"
        ob_spreadsheet_style[f"B{row}"] = "text-align: center; overflow: hidden;"
        ob_spreadsheet_style[f"C{row}"] = "text-align: center; overflow: hidden;"

    ob_spreadsheet.style = ob_spreadsheet_style
    ob_spreadsheet.data = ob_spreadsheet_data
    ob_spreadsheet.headers = ob_spreadsheet_headers
    ob_spreadsheet.meta = ob_spreadsheet_meta
    ob_spreadsheet.width = ob_spreadsheet_width
    ob_spreadsheet.values = ob_spreadsheet_values

    substance_dict["evaporation_temperatures"] = ob_spreadsheet

    utils.create_openbis_object(
        OPENBIS_SESSION,
        type="SUBSTANCE",
        collection=OPENBIS_COLLECTIONS_PATHS["Precursor Substance"],
        props=substance_dict,
    )

    display(Javascript(data="alert('Substance created successfully!')"))
    create_substances_vbox.reset_widgets()

# Upload substances into openBIS

In [ ]:
if OPENBIS_SESSION:
    display(create_substances_vbox)
    display(increase_buttons_size)

home_button.on_click(close_notebook)
save_button.on_click(save_substance)
display(save_close_buttons_hbox)